In [ ]:
# Exemplo retirado de https://www.geeksforgeeks.org/python-arima-model-for-time-series-forecasting/
# importar bibliotecas necessarias
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
filename = io.BytesIO(uploaded['air_passengers.csv'])

In [ ]:
# ler ficheiro csv com dados de passageiros
airline = pd.read_csv(filename, index_col ='TravelDate', parse_dates = True)

In [ ]:
# explorar dataframe
airline.head()
airline.info()
airline.plot()

In [ ]:
# decomposicao ETS (Error, Trend, Seasonality)
result = seasonal_decompose(airline['Passengers'], model ='multiplicative')

In [ ]:
# ETS plot
result.plot()

In [ ]:
!pip install pmdarima # install package pmdarima

In [ ]:
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ajustar funcao auto_arima ao dataset 
stepwise_fit = auto_arima(airline['Passengers'], start_p = 1, start_q = 1,
						max_p = 3, max_q = 3, m = 12,
						start_P = 0, seasonal = True,
						d = None, D = 1, trace = True,
						error_action ='ignore',   # we don't want to know if an order does not work
						suppress_warnings = True, # we don't want convergence warnings
						stepwise = True)		  # set to stepwise

In [ ]:
# imprimir relatorio do ajuste
stepwise_fit.summary()

In [ ]:
#%% dividir em dataset de treino e teste
train = airline.iloc[:len(airline)-12] # retirar os últimos 12 meses do dataset
test = airline.iloc[len(airline)-12:]  # 12 meses para o dataset de teste

In [ ]:
# ajustar SARIMAX(0, 1, 1)x(2, 1, 0, 12) ao dataset de treino
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train['Passengers'],
				order = (0, 1, 1),
				seasonal_order =(2, 1, 0, 12))

result = model.fit()
result.summary()

result.resid.plot(kind='kde')

In [ ]:
#%% calcular previsoes com o modelo ARIMA e comparar com dataset de teste
start = len(train)
end = len(train) + len(test) - 1

In [ ]:
# previsoes para um ano contra dataset de teste
predictions = result.predict(start, end, typ = 'levels').rename("Predictions")

In [ ]:
# plotar previsoes e valores reais do dataset de teste
predictions.plot(legend = True)
test['Passengers'].plot(legend = True)

In [ ]:
#%% avaliar o modelo com MSE e RMSE
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse

In [ ]:
# calcular root mean squared error
rmse(test["Passengers"], predictions)

In [ ]:
# calcular mean squared error
mean_squared_error(test["Passengers"], predictions)

In [ ]:
#%% calcular novas prevsioes como modelo ARIMA
# treinar o modelo com todo o dataset
model = model = SARIMAX(airline['Passengers'],
						order = (0, 1, 1),
						seasonal_order =(2, 1, 0, 12))
result = model.fit()
result.summary() 

In [ ]:
# calcular novas previsoes para os 4 anos seguintes
forecast = result.predict(start = len(airline),
						end = (len(airline)-1) + 4 * 12,
						typ = 'levels').rename('Forecast')

In [ ]:
# plotar previsoes
airline['Passengers'].plot(legend = True)
forecast.plot(legend = True)